In [1]:
# Import libraries
from functions import *

import json
import os
import subprocess
import time

# Creating a P2TR transaction

In this section we'll create a P2TR transaction from scratch in python. This section assumes familiarity with legacy or segwit transactions.

## Reading
- https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki
- https://bitcoinops.org/en/preparing-for-taproot/#from-p2wpkh-to-single-sig-p2tr
- https://bitcoin.stackexchange.com/questions/111098/what-is-the-script-assembly-and-execution-in-p2tr-spend-spend-from-taproot

### Create a P2TR UTXO

In order to create a transaction spending from a P2TR UTXO, we'll first need to create the UTXO that is locked with a p2tr script. To do that, we'll define a private key, use it to generate a pubkey, then convert the pubkey to a p2tr address.

#### Create a P2TR pubkey
So far the public keys we have used in bitcoin were 33 bytes long. The last 32 bytes corresponded to the x coordinate of the point on the curve, and the first byte corresponded to whether the y coordinate was odd or even.

Now with taproot public keys, the keys are just the x coordinate, which makes it only 32 bytes long. To generate a public key from a private key we'll use code from the bip 340 python reference code imported as `tr`. A consequence of this is that every public key has two corresponding private keys.

In [2]:
privkey = bytes.fromhex("1111111111111111111111111111111111111111111111111111111111111111")
pubkey = tr.pubkey_gen(privkey)
print("Pubkey: ", pubkey.hex())
print("Note the length of pubkeys used for taproot: ", len(pubkey))

Pubkey:  4f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa
Note the length of pubkeys used for taproot:  32


In [27]:
# For comparison, here is the function we used to generate pubkeys for legacy and segwit v0 transactions
pubkey_legacy = privkey_to_pubkey(privkey)
print("33 byte legacy/segwitv0 pubkey: ", pubkey_legacy.hex())

# Taproot pubkeys are the same as legacy pubkeys but without the first byte.
assert(pubkey == pubkey_legacy[1:])

33 byte legacy/segwitv0 pubkey:  034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa


#### Tweaking the internal pubkey and secret key

TODO: is this step necessary to create a valid transaction?

#### Convert the pubkey to a scriptPubkey and P2TR address

In [3]:
spk = bytes.fromhex("5120") + pubkey
p2tr_addr = spk_to_bech32(spk, 'regtest')
print("P2TR bech32m address: ", p2tr_addr)

P2TR bech32m address:  bcrt1pfu64hh9hes90w2808n8tjc2ajp5yhddjef0ctx4s7zmsgp6cwx4q344m0j


In [4]:
# Setup bitcoind and fund the address
setup_regtest_bitcoind()
txid_to_spend, index_to_spend = fund_address(p2tr_addr, 2.01)
print(f"UTXO: {txid_to_spend}, {index_to_spend}")

UTXO: 2dc6cf82a4eed6b37af95a98355e8d0a3364952fdd4be08d570e7ae0986f4cdd, 1


## Spending a P2TR UTXO

Now that we have some funds locked up in a p2wpkh utxo, we can create a transaction spending from it. Let's say we want to send 1.5 btc to the address `bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw`. We can decode the scriptPubkey from the address. For more on this step, refer to the previous chapters.

In [5]:
receiver_address = 'bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw'

hrp = 'bcrt'
witver, witprog = bech32.decode(hrp, receiver_address)
pubkey_hash = bytearray(witprog)
receiver_spk = bytes.fromhex("0014") + pubkey_hash

### Create an unsigned P2TR transaction

The first thing we'll do is define the inputs and outputs of our transaction.

In [6]:
# Set our outputs
# Create a new pubkey to use as a change output.
change_privkey = bytes.fromhex("2222222222222222222222222222222222222222222222222222222222222222")
change_pubkey = privkey_to_pubkey(change_privkey)

# Determine our output scriptPubkeys and amounts (in satoshis)
output1_value_sat = int(float("1.5") * 100000000)
output1_spk = receiver_spk
output2_value_sat = int(float("0.5") * 100000000)
output2_spk = bytes.fromhex("0014") + hash160(change_pubkey)

Now that we've defined everything we need, we can fill in the fields we need to create our unsigned transaction. What makes a transaction 'unsigned' is that the witness field is empty. This first step is necessary as the signature will cover the whole transaction (using SIGHASH_ALL). In a later chapter we will cover other sighash types and how they are signed.

In [7]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# OUTPUT 1 
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output1_spk' already defined at the start of the script

# OUTPUT 2
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output2_spk' already defined at the start of the script

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

unsigned_tx:  0200000001dd4c6f98e07a0e578de04bdd2f9564330a8d5e35985af97ab3d6eea482cfc62d0100000000ffffffff0280d1f00800000000160014fc7250a211deddc70ee5a2738de5f07817351cef80f0fa0200000000160014531260aa2a199e228c537dfa42c82bea2c7c1f4d00000000


We can decode this raw transaction to inspect it and see that it has all the information we need apart from the segwit fields (version, marker, and witness).

In [8]:
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + unsigned_tx.hex())
print(decoded)

{
  "txid": "d9a705c4f27cf37fb74b599eebee51e71e8bf0b5d251f32b030186d3c8f16cc1",
  "hash": "d9a705c4f27cf37fb74b599eebee51e71e8bf0b5d251f32b030186d3c8f16cc1",
  "version": 2,
  "size": 113,
  "vsize": 113,
  "weight": 452,
  "locktime": 0,
  "vin": [
    {
      "txid": "2dc6cf82a4eed6b37af95a98355e8d0a3364952fdd4be08d570e7ae0986f4cdd",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 1.50000000,
      "n": 0,
      "scriptPubKey": {
        "asm": "0 fc7250a211deddc70ee5a2738de5f07817351cef",
        "desc": "addr(bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw)#luvpmd0q",
        "hex": "0014fc7250a211deddc70ee5a2738de5f07817351cef",
        "address": "bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw",
        "type": "witness_v0_keyhash"
      }
    },
    {
      "value": 0.50000000,
      "n": 1,
      "scriptPubKey": {
        "asm": "0 531260aa2a199e228c537dfa42c82bea2c7c1f4d",
 

#### Create the sighash
TODO - create the sighash using https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki#common-signature-message

**For now we will use bitcoind to sign our transaction. If you are reading this and are familiar with this step, please let me know!**

In [9]:
# pk_hash = hash160(sender_pubkey)
# scriptcode = bytes.fromhex("76a914" + pk_hash.hex() + "88ac")

# input_amount_sat = int(2.001 * 100_000_000)
# value = input_amount_sat.to_bytes(8, byteorder="little", signed=False)

# hashPrevOuts = hash256(txid + index)
# hashSequence = hash256(sequence)
# hashOutputs = hash256(outputs)
# sighash_type = bytes.fromhex("0100 0000") # SIGHASH_ALL

# tx_digest_preimage = (
#     version
#     + hashPrevOuts
#     + hashSequence
#     + txid
#     + index
#     + varint_len(scriptcode)
#     + scriptcode
#     + value
#     + sequence
#     + hashOutputs
#     + locktime
#     + sighash_type
# )
# print(tx_digest_preimage.hex())

TODO - sign with schnorr signature

In [10]:
# # Create sigHash to be signed
# sighash = hash256(tx_digest_preimage)

# # Sign the sigHash with the input private key
# signing_key = ecdsa.SigningKey.from_string(sender_privkey, curve=ecdsa.SECP256k1) 
# signature = signing_key.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# # Append SIGHASH_ALL to the signature
# signature = signature + bytes.fromhex("01")

# # Witness field
# witness = (
#     # indicate the number of stack items for the txin
#     # 2 items for signature and pubkey
#     bytes.fromhex("02")
#     + pushbytes(signature)
#     + pushbytes(sender_pubkey)
# )

# # tx_in with our new sigScript containing the signature we just created
# inputs_signed = (
#     txid
#     + index
#     + varint_len(scriptsig)
#     + scriptsig
#     + sequence
# )

# # the final signed transaction
# signed_tx = (
#     version
#     + marker
#     + flag
#     + input_count
#     + inputs_signed
#     + output_count
#     + outputs
#     + witness
#     + locktime
# )

# print("signed transaction: ",signed_tx.hex())

In [11]:
# Convert the private key to wallet import format for bitcoind
wif = privkey_to_wif(privkey, compressed_pubkey = True, testnet = True)

out = subprocess.getoutput("bitcoin-cli -regtest signrawtransactionwithkey \"" + unsigned_tx.hex() + "\" '[\"" + wif + "\"]'")
print(out)

{
  "hex": "02000000000101dd4c6f98e07a0e578de04bdd2f9564330a8d5e35985af97ab3d6eea482cfc62d0100000000ffffffff0280d1f00800000000160014fc7250a211deddc70ee5a2738de5f07817351cef80f0fa0200000000160014531260aa2a199e228c537dfa42c82bea2c7c1f4d0140812ce5285b684bff0556f87d3f19fc54c4792e9b81249a8d1fa99ff584b064056eab7213a7e6caf62fe298a7c2b7e0c58bb06b073d7f5f05993df321a31868a800000000",
  "complete": true
}


In [12]:
# save the hex string as signed_tx
j = json.loads(out)
signed_tx = j['hex']

### Broadcast the transaction (on regtest mode)
If we get back a txid (32 byte hash), then it means the tx was successfully broadcast! If we just want to see if the transaction would have been accepted, but without broadcasting it, we can use the `testmempoolaccept` command (commented out).

In [13]:
new_tx_txid = subprocess.getoutput("bitcoin-cli -regtest testmempoolaccept " + "'[\"" +  signed_tx + "\"]'")
print(new_tx_txid)

[
  {
    "txid": "d9a705c4f27cf37fb74b599eebee51e71e8bf0b5d251f32b030186d3c8f16cc1",
    "wtxid": "156e6d9a252857634e3756039fbb37e117d92e36396bade90b9fe8ac1ac7b2b8",
    "allowed": true,
    "vsize": 130,
    "fees": {
      "base": 0.01000000
    }
  }
]


We can decode the serialized transaction using ```decoderawtransaction```. Notice that our output addresses match the change and receiver addresses from earlier.

In [14]:
# print("receiver's p2pkh address: " + receiver_address)
# change_p2pkh_addr = pk_to_p2pkh(change_pubkey, network = "regtest")
# print("sender's change p2pkh address: " + change_p2pkh_addr)

In [15]:
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + signed_tx)
print(decoded)

{
  "txid": "d9a705c4f27cf37fb74b599eebee51e71e8bf0b5d251f32b030186d3c8f16cc1",
  "hash": "156e6d9a252857634e3756039fbb37e117d92e36396bade90b9fe8ac1ac7b2b8",
  "version": 2,
  "size": 181,
  "vsize": 130,
  "weight": 520,
  "locktime": 0,
  "vin": [
    {
      "txid": "2dc6cf82a4eed6b37af95a98355e8d0a3364952fdd4be08d570e7ae0986f4cdd",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "812ce5285b684bff0556f87d3f19fc54c4792e9b81249a8d1fa99ff584b064056eab7213a7e6caf62fe298a7c2b7e0c58bb06b073d7f5f05993df321a31868a8"
      ],
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 1.50000000,
      "n": 0,
      "scriptPubKey": {
        "asm": "0 fc7250a211deddc70ee5a2738de5f07817351cef",
        "desc": "addr(bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw)#luvpmd0q",
        "hex": "0014fc7250a211deddc70ee5a2738de5f07817351cef",
        "address": "bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw",
        "type"

In [16]:
# stop bitcoin core
subprocess.getoutput("bitcoin-cli -regtest stop")

'Bitcoin Core stopping'

## Quiz


 ## Answers
    

## Exercise
